# Databases, SQLAlchemy
## Objective
Build a movie recommendation service

Install `sqlalchemy`

In [ ]:
!conda install sqlalchemy!

## Explore the Open Brewery DB documentation
https://www.openbrewerydb.org/documentation/01-listbreweries#by_city

### Warm Up 1
Write a method search_breweries that searches breweries by some search term.

In [1]:
import requests

## Create our database

In [192]:
import sqlalchemy
sqlalchemy.__version__

'1.3.20'

In [193]:
from sqlalchemy import create_engine

In [203]:
engine = create_engine('sqlite:///openbrewery.db', echo = True)

Check if it exists.

In [204]:
conn = engine.connect()

2021-01-22 17:36:47,800 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-01-22 17:36:47,801 INFO sqlalchemy.engine.base.Engine ()
2021-01-22 17:36:47,803 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-01-22 17:36:47,803 INFO sqlalchemy.engine.base.Engine ()


In [205]:
conn.close()

In [197]:
import os

In [199]:
# os.listdir()

Introspect `engine`

In [201]:
# dir(engine)

List the table names.

In [206]:
engine.table_names()

2021-01-22 17:36:51,251 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-01-22 17:36:51,252 INFO sqlalchemy.engine.base.Engine ()


[]

## Defining a model
Let's define the City model.

[Here is a reference on data types](https://docs.sqlalchemy.org/en/14/core/type_basics.html)

In [210]:
import datetime

In [207]:

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, ForeignKey, Boolean, DateTime


Base = declarative_base()

In [208]:
class City(Base):
    __tablename__ = 'cities'

    city_id = Column(Integer, primary_key=True)
    name = Column(String(100))
    date_created = Column(DateTime, default=datetime.datetime.now)
    
    def __repr__(self):
        return "{}: {}".format(self.city_id, self.name)

Check the tables.

In [209]:
engine.table_names()

2021-01-22 17:40:02,835 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-01-22 17:40:02,837 INFO sqlalchemy.engine.base.Engine ()


[]

What s going on?

In [3]:
Base.metadata.create_all(engine)

In [212]:
engine.table_names()

2021-01-22 17:44:26,678 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-01-22 17:44:26,680 INFO sqlalchemy.engine.base.Engine ()


['cities']

### Exercise
Define the BreweryType model based on the data available and create it.

- name

In [218]:
class BreweryType(Base):
    __tablename__ = 'brewerytypes'

    brewery_type_id = Column(Integer, primary_key=True)
    name = Column(String(100))
    date_created = Column(DateTime, default=datetime.datetime.now)
    
    def __repr__(self):
        return "{}: {}".format(self.zomato_id, self.name)

In [2]:
Base.metadata.create_all(engine)

In [220]:
engine.table_names()

2021-01-22 18:06:05,655 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-01-22 18:06:05,656 INFO sqlalchemy.engine.base.Engine ()


['cities', 'cuisines']

## Creating instances and saving

In [221]:
from sqlalchemy.orm import sessionmaker

In [222]:
Session = sessionmaker(bind=engine)

In [223]:
session = Session()

In [224]:
new_city = City(name='Some city')

In [225]:
session.add(new_city)

In [226]:
our_city = session.query(City).first()

2021-01-22 18:07:29,568 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-01-22 18:07:29,632 INFO sqlalchemy.engine.base.Engine INSERT INTO cities (zomato_id, name, date_created) VALUES (?, ?, ?)
2021-01-22 18:07:29,634 INFO sqlalchemy.engine.base.Engine (99923, 'Some city', '2021-01-22 18:07:29.631859')
2021-01-22 18:07:29,737 INFO sqlalchemy.engine.base.Engine SELECT cities.city_id AS cities_city_id, cities.zomato_id AS cities_zomato_id, cities.name AS cities_name, cities.date_created AS cities_date_created 
FROM cities
 LIMIT ? OFFSET ?
2021-01-22 18:07:29,743 INFO sqlalchemy.engine.base.Engine (1, 0)


In [227]:
our_city.name

'Some city'

### Exercise
Create a script to create cities for all the cities returned by a search query of "ale"

Run it again.

Let's query.

In [235]:
all_cities = session.query(City).all()

2021-01-22 18:19:31,462 INFO sqlalchemy.engine.base.Engine SELECT cuisines.cuisine_id AS cuisines_cuisine_id, cuisines.zomato_id AS cuisines_zomato_id, cuisines.name AS cuisines_name, cuisines.date_created AS cuisines_date_created 
FROM cuisines
2021-01-22 18:19:31,470 INFO sqlalchemy.engine.base.Engine ()


In [236]:
for city in all_cities[:5]:
    print(cuis.name)

Afghan
African
American
Argentine
Armenian


## Querying / Filtering

Let's get order cuisines alphabetically.


In [4]:
for c in session.query(City).order_by(City.name)[:10]:
    print(c)

### Deduping
A common and practical use case.

First, let's get all the distict names.

In [5]:
len(distinct_names)

Let's dedupe.

**Dedupe strategy**
For each name, get all the records that match the name, pick one to be the survivor and delete the others.

- session.delete(obj)
- session.query(City).filter(City.name == name)

In [6]:
distinct_names[0]

Let's find which ones start with "A"

## Mini-Lab 1: Recommendations & Reviews
Let's build a recommendation engine for users.

### Step 1: Populate Brewery Types

### Step 2: Table Creation
There are three new tables we need to create

- Brewery (with the relevant fields)
- User (with username and user_id and favorite type and their city)
- Recommendations (which links to brewery and user)

### Step 3: Create_User
Create a helper function that creates a user and also adds UserCuisine preferences

```python
>>> create_user(username='alberto', favorite_type='..', city='Boston')
```

### Step 4: Simple matchmaking algo
Pick 5 random restaurants that match their preferences and save to the database.

### (Optional) Step 5: Don't repeat the last 3 recommendations
Need to query the DB in the algo.